In [3]:
import pandas as pd
import numpy as np
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#import nltk
#nltk.download('vader_lexicon')

In [4]:
df = pd.read_pickle('data/pickled_NYT.pkl')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 525542 entries, 2007-01-01 to 2016-12-31
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   headline.main     525506 non-null  object
 1   news_desk         356579 non-null  object
 2   section_name      525542 non-null  object
 3   type_of_material  525540 non-null  object
dtypes: object(4)
memory usage: 20.0+ MB


In [10]:
df = df.dropna(subset=['headline.main'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 525506 entries, 2007-01-01 to 2016-12-31
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   headline.main     525506 non-null  object
 1   news_desk         356543 non-null  object
 2   section_name      525506 non-null  object
 3   type_of_material  525504 non-null  object
dtypes: object(4)
memory usage: 20.0+ MB


In [13]:
df['headline.main'][0] + df['headline.main'][1]

'Estimates of Iraqi Civilian DeathsRomania and Bulgaria Join European Union'

In [15]:
df['headline.main'] = df['headline.main'] + ' '
df['headline.main'][0] + df['headline.main'][1]

<ipython-input-15-8a4728c8a5b2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['headline.main'] = df['headline.main'] + ' '


'Estimates of Iraqi Civilian Deaths Romania and Bulgaria Join European Union '

In [18]:
headline_sum = df.groupby('pub_date')['headline.main'].sum()
headline_sum

pub_date
2007-01-01    Estimates of Iraqi Civilian Deaths Romania and...
2007-01-02    For Dodd, Wall Street Looms Large Ford's Lost ...
2007-01-03    Ethics Changes Proposed for House Trips, K Str...
2007-01-04    I Feel Bad About My Face Bush Recycles the Tra...
2007-01-05    Macworld Bingo Anti-Surge Protests Against McC...
                                    ...                        
2016-12-27    Should the U.S. Embassy Be Moved From Tel Aviv...
2016-12-28    When Finding the Right Lawyer Seems Daunting, ...
2016-12-29    Does Empathy Guide or Hinder Moral Action? Chi...
2016-12-30    Shielding Seized Assets From Corruption’s Clut...
2016-12-31    Terrorist Attack at Nightclub in Istanbul Kill...
Name: headline.main, Length: 3651, dtype: object

In [23]:
pd.Timedelta(1, unit='D')

Timedelta('1 days 00:00:00')

In [25]:
headline_sum.index = headline_sum.index + pd.Timedelta(1, unit='D')
headline_sum

pub_date
2007-01-02    Estimates of Iraqi Civilian Deaths Romania and...
2007-01-03    For Dodd, Wall Street Looms Large Ford's Lost ...
2007-01-04    Ethics Changes Proposed for House Trips, K Str...
2007-01-05    I Feel Bad About My Face Bush Recycles the Tra...
2007-01-06    Macworld Bingo Anti-Surge Protests Against McC...
                                    ...                        
2016-12-28    Should the U.S. Embassy Be Moved From Tel Aviv...
2016-12-29    When Finding the Right Lawyer Seems Daunting, ...
2016-12-30    Does Empathy Guide or Hinder Moral Action? Chi...
2016-12-31    Shielding Seized Assets From Corruption’s Clut...
2017-01-01    Terrorist Attack at Nightclub in Istanbul Kill...
Name: headline.main, Length: 3651, dtype: object

In [27]:
sent = SentimentIntensityAnalyzer()

In [29]:
sent.polarity_scores(headline_sum[0])

{'neg': 0.16, 'neu': 0.738, 'pos': 0.102, 'compound': -0.9851}

In [35]:
sent_head = headline_sum.apply(sent.polarity_scores)
sent_head

pub_date
2007-01-02    {'neg': 0.16, 'neu': 0.738, 'pos': 0.102, 'com...
2007-01-03    {'neg': 0.119, 'neu': 0.772, 'pos': 0.109, 'co...
2007-01-04    {'neg': 0.192, 'neu': 0.744, 'pos': 0.064, 'co...
2007-01-05    {'neg': 0.114, 'neu': 0.823, 'pos': 0.063, 'co...
2007-01-06    {'neg': 0.083, 'neu': 0.827, 'pos': 0.09, 'com...
                                    ...                        
2016-12-28    {'neg': 0.15, 'neu': 0.748, 'pos': 0.102, 'com...
2016-12-29    {'neg': 0.116, 'neu': 0.78, 'pos': 0.104, 'com...
2016-12-30    {'neg': 0.145, 'neu': 0.755, 'pos': 0.1, 'comp...
2016-12-31    {'neg': 0.167, 'neu': 0.741, 'pos': 0.093, 'co...
2017-01-01    {'neg': 0.155, 'neu': 0.683, 'pos': 0.162, 'co...
Name: headline.main, Length: 3651, dtype: object

In [38]:
sent_head = pd.DataFrame.from_records(sent_head.values, index=sent_head.index)
sent_head

,neg,neu,pos,compound
pub_date,,,,
2007-01-02,0.160,0.738,0.102,-0.9851
2007-01-03,0.119,0.772,0.109,-0.4451
2007-01-04,0.192,0.744,0.064,-0.9994
2007-01-05,0.114,0.823,0.063,-0.9972
2007-01-06,0.083,0.827,0.090,0.9720
...,...,...,...,...
2016-12-28,0.150,0.748,0.102,-0.9845
2016-12-29,0.116,0.780,0.104,-0.3421
2016-12-30,0.145,0.755,0.100,-0.9863


In [117]:
main_df = pd.read_csv('data/DJIA_adj_ver01.csv')
main_df.head()

,Date,Close,Volume
0,2006-12-28,12501.519531,126740000.0
1,2006-12-29,12463.150391,161560000.0
2,2006-12-30,12465.424219,194688000.0
3,2006-12-31,12467.698047,227816000.0
4,2007-01-01,12469.971875,260944000.0


In [118]:
main_df['Date'] = pd.to_datetime(main_df['Date'])
main_df = main_df.set_index('Date')
main_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3657 entries, 2006-12-28 to 2016-12-31
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   3657 non-null   float64
 1   Volume  3657 non-null   float64
dtypes: float64(2)
memory usage: 85.7 KB


In [119]:
main_df = main_df.join(sent_head, how='left')
main_df.head(10)

,Close,Volume,neg,neu,pos,compound
Date,,,,,,
2006-12-28,12501.519531,1.267400e+08,NaN,NaN,NaN,NaN
2006-12-29,12463.150391,1.615600e+08,NaN,NaN,NaN,NaN
2006-12-30,12465.424219,1.946880e+08,NaN,NaN,NaN,NaN
2006-12-31,12467.698047,2.278160e+08,NaN,NaN,NaN,NaN
2007-01-01,12469.971875,2.609440e+08,NaN,NaN,NaN,NaN
2007-01-02,12472.245703,2.940720e+08,0.160,0.738,0.102,-0.9851
2007-01-03,12474.519531,3.272000e+08,0.119,0.772,0.109,-0.4451
2007-01-04,12480.690430,2.590600e+08,0.192,0.744,0.064,-0.9994
2007-01-05,12398.009766,2.352200e+08,0.114,0.823,0.063,-0.9972


In [120]:
main_df.Close = main_df.Close.astype('int')
main_df.head()

,Close,Volume,neg,neu,pos,compound
Date,,,,,,
2006-12-28,12501,126740000.0,NaN,NaN,NaN,NaN
2006-12-29,12463,161560000.0,NaN,NaN,NaN,NaN
2006-12-30,12465,194688000.0,NaN,NaN,NaN,NaN
2006-12-31,12467,227816000.0,NaN,NaN,NaN,NaN
2007-01-01,12469,260944000.0,NaN,NaN,NaN,NaN


In [121]:
main_df.head(10)

,Close,Volume,neg,neu,pos,compound
Date,,,,,,
2006-12-28,12501,1.267400e+08,NaN,NaN,NaN,NaN
2006-12-29,12463,1.615600e+08,NaN,NaN,NaN,NaN
2006-12-30,12465,1.946880e+08,NaN,NaN,NaN,NaN
2006-12-31,12467,2.278160e+08,NaN,NaN,NaN,NaN
2007-01-01,12469,2.609440e+08,NaN,NaN,NaN,NaN
2007-01-02,12472,2.940720e+08,0.160,0.738,0.102,-0.9851
2007-01-03,12474,3.272000e+08,0.119,0.772,0.109,-0.4451
2007-01-04,12480,2.590600e+08,0.192,0.744,0.064,-0.9994
2007-01-05,12398,2.352200e+08,0.114,0.823,0.063,-0.9972


In [122]:
main_df.Close

Date
2006-12-28    12501
2006-12-29    12463
2006-12-30    12465
2006-12-31    12467
2007-01-01    12469
              ...  
2016-12-27    19945
2016-12-28    19833
2016-12-29    19819
2016-12-30    19819
2016-12-31    19819
Name: Close, Length: 3657, dtype: int32

In [123]:
temp = main_df[['Close']].copy()
temp.index = temp.index + pd.Timedelta(1, unit='D')
temp.columns = ['t_1']
main_df = main_df.join(temp, how='left')

In [124]:
temp = main_df[['Close']].copy()
temp.index = temp.index + pd.Timedelta(2, unit='D')
temp.columns = ['t_2']
main_df = main_df.join(temp, how='left')

In [125]:
temp = main_df[['Close']].copy()
temp.index = temp.index + pd.Timedelta(3, unit='D')
temp.columns = ['t_3']
main_df = main_df.join(temp, how='left')

In [126]:
temp = main_df[['Close']].copy()
temp.index = temp.index + pd.Timedelta(4, unit='D')
temp.columns = ['t_4']
main_df = main_df.join(temp, how='left')

In [127]:
temp = main_df[['Close']].copy()
temp.index = temp.index + pd.Timedelta(5, unit='D')
temp.columns = ['t_5']
main_df = main_df.join(temp, how='left')
main_df

,Close,Volume,neg,neu,pos,compound,t_1,t_2,t_3,t_4,t_5
Date,,,,,,,,,,,
2006-12-28,12501,126740000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006-12-29,12463,161560000.0,NaN,NaN,NaN,NaN,12501.0,NaN,NaN,NaN,NaN
2006-12-30,12465,194688000.0,NaN,NaN,NaN,NaN,12463.0,12501.0,NaN,NaN,NaN
2006-12-31,12467,227816000.0,NaN,NaN,NaN,NaN,12465.0,12463.0,12501.0,NaN,NaN
2007-01-01,12469,260944000.0,NaN,NaN,NaN,NaN,12467.0,12465.0,12463.0,12501.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2016-12-27,19945,158540000.0,0.160,0.743,0.097,-0.9837,19942.0,19939.0,19936.0,19933.0,19918.0
2016-12-28,19833,188350000.0,0.150,0.748,0.102,-0.9845,19945.0,19942.0,19939.0,19936.0,19933.0
2016-12-29,19819,172040000.0,0.116,0.780,0.104,-0.3421,19833.0,19945.0,19942.0,19939.0,19936.0


In [128]:
main_df.to_pickle('data/main_df_ver01.pkl')